# Simulation pipeline

This is the pipeline version of simulation and data analysis of gene-based CNV analysis methods.

In [ ]:
%set -j 4
%run simulate
[simulate: shared = {'simulate_data': 'output'}]
parameter: gamma = [(None, None), (3,1), (5,1)]
parameter: sample_size = [50, 100, 500]
parameter: n_causal_gene = [50, 100]
parameter: seed = [999]
parameter: name = 'data/del_sim'
input: for_each = ['gamma', 'sample_size', 'n_causal_gene', 'seed']
output: "${name}_%s.pkl" % "scale{}_shape{}_N{}_M{}".format("${_gamma[0]}", "${_gamma[1]}", "${_sample_size}", "${_n_causal_gene}", "${_seed}")
task:
python:
import sys, os
sys.path.append(os.path.expanduser('~/GIT/cnv-gene-mapping/analysis'))
from simulation import *
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['n_case'] = args['n_ctrl'] = ${_sample_size}
args['n_causal_gene'] = ${_n_causal_gene}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, ${_index})

Set sos options to "-j 4"


,e88151fc6fbc45fa3068fb4160e3f299


,04d4faa702537e7bb1fd6a85d8b7ecc4


,105e6f0f40abaee394ad86b5c64d6bc9


,29ff60815ce0c1e7be526ecb26bdd0c0


...

,39b22431d867cd8fcedd18a44d741f37


,eef15edc907ac42d402112a83566a92d


,32b2dafb601dd01f4efefa5a76cb596e


,49b3160710cf87ef095133f3af2fbeb3


,9e133a517e0c71c86f6683fcf216cc60
